# 1. Q&A Annotation for Events

In [7]:
import json
import random
import csv
import re

templates = {
    "event_info": [
        {
            "template": "What event is happening at {venue} on {event_time} organized by {organization}?",
            "answer_field": "event_name"
        },
        {
            "template": "Who is organizing the event '{event_name}'?",
            "answer_field": "organization"
        },
        {
            "template": "Which organization is hosting '{event_name}'?",
            "answer_field": "organization"
        },
        {
            "template": "Where is the event '{event_name}' held?",
            "answer_field": "venue"
        },
        {
            "template": "When is the event '{event_name}' scheduled to take place?",
            "answer_field": "event_time"
        },
        {
            "template": "What is the date of the event '{event_name}'?",
            "answer_field": "event_time"
        },
        {
            "template": "What time is the event '{event_name}' at {venue} organized by {organization}?",
            "answer_field": "event_time"
        },
        {
            "template": "Which event is taking place at {venue} organized by {organization} on {event_time}?",
            "answer_field": "event_name"
        },
        {
            "template": "What is the name of the event organized by {organization} at {venue} on {event_time}?",
            "answer_field": "event_name"
        },
        {
            "template": "What event is scheduled at {venue} on {event_time}?",
            "answer_field": "event_name"
        },
        {
            "template": "Which event is happening at {venue} on {event_time}?",
            "answer_field": "event_name"
        }
    ],
    "venue_related_questions": [
        {
            "template": "At which venue will '{event_name}' take place?",
            "answer_field": "venue"
        },
        {
            "template": "What is the main venue for events organized by {organization}?",
            "answer_field": "venue"
        },
        {
            "template": "Which organization is hosting events at {venue}?",
            "answer_field": "organization"
        }
    ]
}

def format_date_variations(event_time):
    single_date_pattern = r"\b\w{3}, \w{3} \d{1,2}, \d{4}\b"
    range_date_pattern = r"\b\w{3}, \w{3} \d{1,2} - \w{3}, \w{3} \d{1,2}, \d{4}\b"

    formats = []

    if re.match(single_date_pattern, event_time):
        date_obj = re.findall(r'\w{3} \d{1,2}, \d{4}', event_time)
        formats = [
            event_time.strip(),
            date_obj[0].replace("2025", "'25").strip(),  
            date_obj[0].replace("Oct", "October").strip(),  
            date_obj[0].replace(",", "").strip(), 
            f"{date_obj[0].split(' ')[0]} {date_obj[0].split(' ')[1]}".strip(),  # 月 日
        ]
    elif re.match(range_date_pattern, event_time):
        start_date, end_date = event_time.split("-")
        start_formats = format_date_variations(start_date.strip()) 
        end_formats = format_date_variations(end_date.strip())     
        
        for start in start_formats:
            for end in end_formats:
                formats.append(f"{start.strip()} - {end.strip()}")

    return ";".join(formats).rstrip(',')

def generate_questions_from_json(data, templates):
    questions_and_answers = []
    for item in data:
        date_variations = format_date_variations(item["event_time"])

        for category, template_list in templates.items():
            for template_dict in template_list:
                template = template_dict["template"]
                answer_field = template_dict["answer_field"]
                
                if answer_field not in item:
                    continue 

                other_events = [d["event_name"] for d in data if d != item]
                other_venues = [d["venue"] for d in data if d != item]
                other_event = random.choice(other_events) if other_events else "N/A"
                other_venue = random.choice(other_venues) if other_venues else "N/A"

                try:
                    question = template.format(
                        event_name=item["event_name"],
                        event_time=item["event_time"],
                        venue=item["venue"],
                        organization=item["organization"],
                        other_event=other_event,
                        other_venue=other_venue
                    )
                except KeyError as e:
                    print(f"Missing key {e} in item: {item}")
                    continue 

                if answer_field == "event_time":
                    answer = date_variations 
                else:
                    answer = item[answer_field]

                questions_and_answers.append((question, answer))
    return questions_and_answers

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)
    
def write_to_csv(file_name, questions_and_answers):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Time Sensitive Label", "Question", "Reference Answer"])
    
        for question, answer in questions_and_answers:
            writer.writerow(["", question, answer])

def main():
    file_path = "../raw_documents/Music_Culture/1_Symphony_Events.json"
    json_data = load_json_data(file_path)
    
    questions_and_answers = generate_questions_from_json(json_data, templates)
    output_file = "../../QA/Music_Symphony_Events.csv"
    write_to_csv(output_file, questions_and_answers)
    print(f"Generated {len(questions_and_answers)} questions and answers, saved to {output_file}")

if __name__ == "__main__":
    main()

Generated 784 questions and answers, saved to ../../QA/Music_Symphony_Events.csv
